In [1]:
!nvidia-smi

Wed Nov  3 07:17:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
# DON"T USE WANDB, theres a bug with it handling distributed training
#!pip uninstall -y wandb
!pip install --upgrade wandb

#INSTALLING WANDB ALTERNATIVE
#!pip install comet_ml

## INSTALLING TRANSFORMERS FROM SOURCE
!pip install git+https://github.com/huggingface/transformers.git@master
#!pip install --upgrade transformers

!pip install --upgrade numpy tokenizers rouge_score tqdm
!pip install datasets

## INSTALLING PyTorch/XLA using pip
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

## INSTALLING PyTorch/XLA from source
#!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --version 1.9 --apt-packages libomp5 libopenblas-dev

#!pip install git+https://github.com/pytorch/pytorch.git@master
#!pip install --upgrade torch

#!pip install --upgrade deepspeed
!pip install git+https://github.com/microsoft/DeepSpeed.git@master

     |████████████████████████████████| 1.7 MB 896 kB/s eta 0:00:01
  Attempting uninstall: wandb
    Found existing installation: wandb 0.10.33
    Uninstalling wandb-0.10.33:
      Successfully uninstalled wandb-0.10.33
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.5.0 requires wandb<0.11.0,>=0.10.0, but you have wandb 0.12.6 which is incompatible.
  Cloning https://github.com/huggingface/transformers.git (to revision master) to /tmp/pip-req-build-vntxnbo8
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-vntxnbo8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 59 kB 818 kB/s eta 0:00:01
  Created wheel for transformers: filename=transformers-4.13.0.dev0-py3-none-any.whl size=3077935

In [2]:
from subprocess import run
USE_DEEPSPEED_OR_TPU = "DEEPSPEED" ## OPTIONS: "DEEPSPEED", "TPU", "NONE"
PY_TPU_VERSION = "1.10"
MONITORING_SYSTEM = "Wandb" ## OPTIONS: "Wandb", "Cometml"

### INITIALIZING ENVIRONMENT VARIABLES FOR TPU OR DEEPSPEED:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

if USE_DEEPSPEED_OR_TPU=="TPU":
    os.environ["XLA_USE_BF16"] = "1"
    
elif USE_DEEPSPEED_OR_TPU=="DEEPSPEED":
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '9994' # modify if RuntimeError: Address already in use
    os.environ['RANK'] = "0"
    os.environ['LOCAL_RANK'] = "0"
    os.environ['WORLD_SIZE'] = "1"

import gc

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
huggingface_hub_token = user_secrets.get_secret("huggingface-hub-token")
os.environ["WANDB_API_KEY"] = user_secrets.get_secret("wandb-key")
os.environ["COMET_API_KEY"] = user_secrets.get_secret("comet-key")

if MONITORING_SYSTEM == "Cometml":
    import comet_ml
else:
    import wandb
from tqdm.notebook import tqdm
import transformers
import tokenizers
#import datasets
import tensorflow as tf
import glob
import numpy as np
import json
import pandas as pd
from functools import partial

from datasets import (
    load_dataset,
    load_metric,
    load_from_disk,
)

from transformers import(
    Trainer,
    TrainingArguments,
    is_torch_tpu_available,
    AutoTokenizer,
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoModel,
    DataCollatorWithPadding,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
from transformers.modeling_outputs import Seq2SeqLMOutput
from transformers.optimization import (
    Adafactor,
    AdafactorSchedule,
    get_cosine_schedule_with_warmup,
    get_linear_schedule_with_warmup,
)

from dataclasses import dataclass, field
import nltk
nltk.download("punkt")

import torch
from torch.utils.data import Dataset
import torch.nn as nn

from kaggle_datasets import KaggleDatasets

if is_torch_tpu_available():
    import torch_xla.core.xla_model as xm
    import torch_xla.distributed.parallel_loader as pl
    import torch_xla.distributed.xla_multiprocessing as xmp
    import torch_xla.utils.serialization as xser
else:
    import deepspeed

from typing import Dict, List, Optional

from subprocess import run


rouge_metric = load_metric("rouge")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

In [3]:
from platform import python_version
print(python_version())
print(tf.__version__)
print(transformers.__version__)
print(torch.__version__)

if not is_torch_tpu_available():
    print(deepspeed.__version__)

3.7.10
2.4.1
4.13.0.dev0
1.7.0
0.5.6+426dd2b


In [4]:
TPU_CORES=0
N_GPUS=0

if is_torch_tpu_available():
    TPU_CORES = 8
    TRAIN_BATCH_SIZE = 32
    VALID_BATCH_SIZE = 32
    GRAD_ACCU_STEPS = 1
    FP16 = False
    DS_Config=None
elif USE_DEEPSPEED_OR_TPU=="DEEPSPEED":
    N_GPUS = 1
    TRAIN_BATCH_SIZE = 10
    VALID_BATCH_SIZE = 10
    GRAD_ACCU_STEPS = 1
    FP16=True
    DS_Config="zero3"
    #DS_Config="zero3_infinity"
    #DS_Config="zero2"
else:
    N_GPUS = 1
    TRAIN_BATCH_SIZE = 4
    VALID_BATCH_SIZE = 4
    GRAD_ACCU_STEPS = 1
    FP16=True
    DS_Config=None
    
class ConfigInitializer:
    
    ## DATA CONFIGS
    use_fast_tokenizer=True
    #prefix="summarize: "
    prefix=""
    
    valid_test_tpu_support=True
    train_required_columns=["input_ids", "attention_mask",
                            "decoder_attention_mask",
                            "labels"]
    eval_required_columns = ["input_ids", "attention_mask", 
                             "decoder_attention_mask",
                             "labels"]
    max_input_len=256
    max_output_len=64
    truncation=True
    padding="max_length"
    return_tensors="pt"
    input_column="document"
    output_column="summary"

    train_batch_size=TRAIN_BATCH_SIZE
    valid_batch_size=VALID_BATCH_SIZE
    test_batch_size=4

    drop_last_train=True
    drop_last_valid=False
    drop_last_test=False

    num_workers_train=2
    num_workers_valid=2
    num_workers_test=2

    datasets_main_dir=os.path.join(os.curdir, "my_datasets")
    train_ds_path=os.path.join(datasets_main_dir, "train_ds.pt")
    valid_ds_path=os.path.join(datasets_main_dir, "valid_ds.pt")
    test_ds_path=os.path.join(datasets_main_dir, "test_ds.pt")
    
    ## TRAINING CONFIGS
    max_steps=1000
    optimizer_scale_parameter=False
    optimizer_relative_step=False
    optimizer_warmup_init=False
    lr=1e-3
    initial_lr=1e-3
    epochs=1
    logging_ratio = 0.01
    warmup_ratio=0.1
    saving_ratio = 0.2
    eval_ratio = 0.1
    total_train_steps=100
    logging_steps=None
    eval_steps=None
    save_steps=None
    save_strategy="no"
    evaluation_strategy="steps"
    output_dir=os.path.join(os.curdir, "results")
    logging_dir=os.path.join(os.curdir, "log")
    weight_decay=0.0
    num_beams=4
    skip_special_tokens_in_decode=True
    use_stemmer_for_compute=True
    adam_epsilon=1e-8
    gradient_accumulation_steps=GRAD_ACCU_STEPS
    n_gpu=N_GPUS
    tpu_cores=TPU_CORES
    seed=42
    use_ada_factor=True
    compute_metrics=False
    fp16=FP16
    rouge_before_training=False
    limited_host_mem=False
    use_ds_optimizer_scheduler=False
    
    def __init__(self, gcs_path, project_name, model_checkpoint,
                 model_name,dataset_name, dataset_dir, csv_folder=None, train_len=None,
                 valid_len=200, test_len=20, monitoring_system="Wandb"):
        self.dataset_name = dataset_name
        self.project_name = project_name

        if csv_folder:
            self.train_files = os.path.join(gcs_path, "Sci_Summarizer_Data", dataset_dir, csv_folder, f"{dataset_name}_train.csv")
            self.valid_files = os.path.join(gcs_path, "Sci_Summarizer_Data", dataset_dir, csv_folder, f"{dataset_name}_valid.csv")
            self.test_files = os.path.join(gcs_path, "Sci_Summarizer_Data", dataset_dir, csv_folder, f"{dataset_name}_test.csv")
        else:
            self.train_files = os.path.join(gcs_path, "Sci_Summarizer_Data", dataset_dir, f"{dataset_name}_train.csv")
            self.valid_files = os.path.join(gcs_path, "Sci_Summarizer_Data", dataset_dir, f"{dataset_name}_valid.csv")
            self.test_files = os.path.join(gcs_path, "Sci_Summarizer_Data", dataset_dir, f"{dataset_name}_test.csv")

        self.model_checkpoint = model_checkpoint
        self.model_name = model_name

        self.xser_save_path=f"{model_name.replace('-', '_')}_{dataset_name}"
        self.config_json_path=f"{self.xser_save_path}.json"
        self.tokenizer_save_path=f"{self.xser_save_path}_tokenizer"

        train_file_path = os.path.join(os.curdir,f"{self.dataset_name}_train.csv")
        valid_file_path = os.path.join(os.curdir,f"{self.dataset_name}_valid.csv")
        test_file_path  = os.path.join(os.curdir,f"{self.dataset_name}_test.csv")
        
        train_file_check = os.path.exists(train_file_path)
        valid_file_check = os.path.exists(valid_file_path)
        test_file_check = os.path.exists(test_file_path)
        
        file_check_list = [(self.train_files, train_file_check),
                           (self.valid_files, valid_file_check),
                           (self.test_files, test_file_check)]

        for data_file, file_check in file_check_list:
            if not file_check:
                run(["gsutil", "cp", data_file, "."])

        self.train_files = os.path.join(os.curdir,f"{self.dataset_name}_train.csv")
        self.valid_files = os.path.join(os.curdir,f"{self.dataset_name}_valid.csv")
        self.test_files = os.path.join(os.curdir,f"{self.dataset_name}_test.csv")
        

        self.train_len=train_len
        self.valid_len=valid_len
        self.test_len=test_len
        self.total_train_step=None
        
        if DS_Config:
            self.deepspeed_config = os.path.join(os.curdir, f"ds_config_{DS_Config}.json")
        else:
            self.deepspeed_config = DS_Config

        if monitoring_system == "Cometml":
            comet_ml.init(project_name=self.project_name)
        else:
            pass
            #wandb.init(project=self.project_name)

## DeepSpeed Config files:

    ## If you don't want to use AdaFactor then, copy the below dict into config json
    
    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": "auto",
            "betas": "auto",
            "eps": "auto",
            "weight_decay": "auto"
        }
    },

    "scheduler": {
        "type": "WarmupLR",
        "params": {
            "warmup_min_lr": "auto",
            "warmup_max_lr": "auto",
            "warmup_num_steps": "auto"
        }
    },
    
    ## And remove the below line from config json:
    
    "zero_allow_untested_optimizer": true

### Zero2:

    zero2_config_dict = {
        "fp16": {
            "enabled": "auto",
            "loss_scale": 0,
            "loss_scale_window": 1000,
            "initial_scale_power": 16,
            "hysteresis": 2,
            "min_loss_scale": 1
        },

        "zero_allow_untested_optimizer": True,

        """
        "optimizer": {
            "type": "AdamW",
            "params": {
                "lr": "auto",
                "betas": "auto",
                "eps": "auto",
                "weight_decay": "auto"
            }
        },
        """
        """
        "scheduler": {
            "type": "WarmupLR",
            "params": {
                "warmup_min_lr": "auto",
                "warmup_max_lr": "auto",
                "warmup_num_steps": "auto"
            }
        },
        """

        "zero_optimization": {
            "stage": 2,
            """
            "offload_optimizer": {
                "device": None,
                "pin_memory": True
            },
            """
            "allgather_partitions": True,
            "allgather_bucket_size": 5e8,
            "overlap_comm": True,
            "reduce_scatter": True,
            "reduce_bucket_size": 5e8,
            "contiguous_gradients": True
        },

        "gradient_accumulation_steps": "auto",
        "gradient_clipping": "auto",
        "steps_per_print": 2000,
        "train_batch_size": "auto",
        "train_micro_batch_size_per_gpu": TRAIN_BATCH_SIZE,
        "wall_clock_breakdown": False
    }

    with open("ds_config_zero2.json", "w") as config_file:
        json.dump(zero2_config_dict, config_file)

        "offload_optimizer": {
            "device": "cpu",
            "pin_memory": true
        },
        "offload_param": {
            "device": "cpu",
            "pin_memory": true
        },

In [5]:
zero3_config_dict = {
    "fp16": {
        "enabled": "auto",
        "loss_scale": 0,
        "loss_scale_window": 1000,
        "initial_scale_power": 16,
        "hysteresis": 2,
        "min_loss_scale": 1
    },
    
    "zero_allow_untested_optimizer": True,

    """
    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": "auto",
            "betas": "auto",
            "eps": "auto",
            "weight_decay": "auto"
        }
    },
    """
    """
    "scheduler": {
        "type": "WarmupLR",
        "params": {
            "warmup_min_lr": "auto",
            "warmup_max_lr": "auto",
            "warmup_num_steps": "auto"
        }
    },
    """
    
    "zero_optimization": {
        "stage": 3,
        "offload_optimizer": {
            "device": "cpu",
            "pin_memory": False
        },
        "offload_param": {
            "device": "cpu",
            "pin_memory": True
        },
        "overlap_comm": False,
        "contiguous_gradients": True,
        "sub_group_size": 1e5,
        "allgather_bucket_size": 1e8,
        "reduce_bucket_size": "auto",
        "stage3_prefetch_bucket_size": "auto",
        "stage3_param_persistence_threshold": "auto",
        "stage3_max_live_parameters": 1e5,
        "stage3_max_reuse_distance": 1e5,
        "stage3_gather_fp16_weights_on_model_save": True
    },
    
    "gradient_accumulation_steps": "auto",
    "gradient_clipping": "auto",
    "steps_per_print": 2000,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "wall_clock_breakdown": False
}


with open("ds_config_zero3.json", "w") as config_file:
    json.dump(zero3_config_dict, config_file)

### Zero3-infinity:

In [6]:
zero3_infinity_config_dict = {
    "fp16": {
        "enabled": "auto",
        "loss_scale": 0,
        "loss_scale_window": 1000,
        "initial_scale_power": 16,
        "hysteresis": 2,
        "min_loss_scale": 1
    },
    
    "zero_allow_untested_optimizer": True,
    
    "zero_optimization": {
        "stage": 3,
        "offload_optimizer": {
            "device": "nvme",
            "nvme_path": "/local_nvme",
            "pin_memory": True,
            "buffer_count": 4,
            "fast_init": False
        },
        "offload_param": {
            "device": "nvme",
            "nvme_path": "/local_nvme",
            "pin_memory": True,
            "buffer_count": 5,
            "buffer_size": 1e8,
            "max_in_cpu": 1e9
        },
        "aio": {
            "block_size": 262144,
            "queue_depth": 32,
            "thread_count": 1,
            "single_submit": False,
            "overlap_events": True
        },
        "overlap_comm": True,
        "contiguous_gradients": True,
        "sub_group_size": 1e9,
        "reduce_bucket_size": "auto",
        "stage3_prefetch_bucket_size": "auto",
        "stage3_param_persistence_threshold": "auto",
        "stage3_max_live_parameters": 1e9,
        "stage3_max_reuse_distance": 1e9,
        "stage3_gather_fp16_weights_on_model_save": True
    },

    "gradient_accumulation_steps": "auto",
    "gradient_clipping": "auto",
    "steps_per_print": 2000,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "wall_clock_breakdown": False
}

with open("ds_config_zero3_infinity.json", "w") as config_file:
    json.dump(zero3_infinity_config_dict, config_file)

In [7]:
GCS_PATH = KaggleDatasets().get_gcs_path("sci-summarizer-data")
print(GCS_PATH)

Config = ConfigInitializer(project_name="t5_large_xsum",
                           monitoring_system=MONITORING_SYSTEM,
                           #model_checkpoint="microsoft/prophetnet-large-uncased-cnndm",
                           #model_checkpoint="google/pegasus-cnn_dailymail",
                           #model_checkpoint="facebook/bart-large-cnn",
                           model_checkpoint="t5-large",
                           #model_checkpoint="t5-base",
                           #model_checkpoint="t5-small",
                           
                           #model_name="prophetnet",
                           #model_name="pegasus",
                           #model_name="bart-large",
                           #model_name="bart",
                           model_name="t5-large",
                           #model_name="t5-base",
                           #model_name="t5-small",
                           
                           dataset_name="xsum",
                           gcs_path=GCS_PATH,
                           dataset_dir="xsum_data",
                           train_len=None,
                           valid_len=200,
                           test_len=20)

gs://kds-6abe672a34848b32096cc0bc1d10fef197e6f9ee6df5b1391c6a7bf9


In [8]:
def compute_rouge(model, tokenizer,
                  tokenized_dataset,
                  gen_kwargs=None,
                  Config=Config):
    
    def postprocess_text(preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [label.strip() for label in labels]

        # rougeLSum expects newline after each sentence
        preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
        labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

        return preds, labels
    

    for batch in tqdm(tokenized_dataset,
                      total=len(tokenized_dataset),
                      unit="batchs"):
        
        if is_torch_tpu_available():
            temp_batch = {
                "input_ids": batch["input_ids"],
                "attention_mask": batch["attention_mask"],
            }
        else:
            temp_batch={
                "input_ids": batch["input_ids"].to("cuda"),
                "attention_mask": batch["attention_mask"].to("cuda"),
            }
            
        labels = batch["labels"]
        temp_batch.update(gen_kwargs)
        generated_tokens = model.generate(**temp_batch)

        if isinstance(generated_tokens, tuple):
            generated_tokens = generated_tokens[0]


        decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=Config.skip_special_tokens_in_decode)
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=Config.skip_special_tokens_in_decode)
        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

        rouge_metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    result = rouge_metric.compute(use_stemmer=Config.use_stemmer_for_compute)

    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    result = {k: round(v, 4) for k, v in result.items()}
    return result

# Initializing the tokenizer and tokenizing the data:

In [9]:
tokenizer = AutoTokenizer.from_pretrained(Config.model_checkpoint, use_fast=Config.use_fast_tokenizer)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [10]:
class MyXSum(Dataset):
    
    def __init__(self, Config, tokenizer, main_ds, split_type):  
        
        self.model_name = Config.model_checkpoint
        self.dataset = main_ds[split_type]
        self.tokenizer = tokenizer
        self.Config = Config
        
        if split_type in set(["validation", "test"]):
            self.required_columns = Config.eval_required_columns
            if split_type == "validation":
                num_samples = Config.valid_len
            else:
                num_samples = Config.test_len
        else:
            self.required_columns = Config.train_required_columns
            num_samples = Config.train_len
            
        if num_samples:
            self.dataset = self.dataset.select(list(range(0, num_samples)))
  
    def __len__(self):
        return self.dataset.shape[0]
    
    def preprocess_function(self, examples):

        _inputs = [self.Config.prefix + examples[self.Config.input_column]]
        _target = [examples[self.Config.output_column]]
        
        model_inputs = self.tokenizer(_inputs, max_length=self.Config.max_input_len,
                                      truncation=self.Config.truncation, padding=self.Config.padding,
                                      return_tensors=self.Config.return_tensors)

        # Setup the tokenizer for targets
        with self.tokenizer.as_target_tokenizer():
            labels = self.tokenizer(_target, max_length=self.Config.max_output_len,
                                    truncation=self.Config.truncation, padding=self.Config.padding,
                                    return_tensors=self.Config.return_tensors)
            


        model_inputs = {
            "input_ids": model_inputs["input_ids"].squeeze(),
            "attention_mask": model_inputs["attention_mask"].squeeze(),
            "decoder_input_ids": labels["input_ids"].squeeze(),
            "decoder_attention_mask": labels["attention_mask"].squeeze(),
            "labels": labels["input_ids"].squeeze(),
        }
        
        model_inputs = {k: model_inputs[k] for k in self.required_columns}
        
        return model_inputs
    
  
    def __getitem__(self, index):

        return self.preprocess_function(self.dataset[index])

In [11]:
def get_datasets(Config, main_dataset, tokenizer):
    
    train_ds = MyXSum(Config, tokenizer, main_dataset, "train")
    valid_ds = MyXSum(Config, tokenizer, main_dataset, "validation")
    test_ds = MyXSum(Config, tokenizer, main_dataset, "test")
    
    data_dict = {
        "train": train_ds,
        "validation": valid_ds,
        "test": test_ds,
    }
    return data_dict 

In [12]:
xsum_ds = load_dataset("csv", data_files={"train": Config.train_files,
                                          "validation": Config.valid_files,
                                          "test": Config.test_files})

os.makedirs(Config.datasets_main_dir, exist_ok=True)

data_dict = get_datasets(Config, xsum_ds, tokenizer)

torch.save(data_dict["train"], Config.train_ds_path)
torch.save(data_dict["validation"], Config.valid_ds_path)
torch.save(data_dict["test"], Config.test_ds_path)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-7dc108abe4d1fe94/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
del xsum_ds
del data_dict
gc.collect()

89

# Initializing the Model: 

In [14]:
@dataclass
class T2TDataCollator(DataCollatorWithPadding):
    
    def collate_batch(self, batch: List) -> Dict[str, torch.Tensor]:
        """
        Take a list of samples from a Dataset and collate them into a batch.
        Returns:
            A dictionary of tensors
        """
        input_ids = torch.stack([example['input_ids'] for example in batch])
        labels = torch.stack([example['decoder_input_ids'] for example in batch])
        labels[labels[:, :] == 0] = -100
        attention_mask = torch.stack([example['attention_mask'] for example in batch])
        decoder_attention_mask = torch.stack([example['decoder_attention_mask'] for example in batch])
        

        return {
            'input_ids': input_ids.squeeze(), 
            'attention_mask': attention_mask.squeeze(),
            'labels': labels.squeeze(), 
            'decoder_attention_mask': decoder_attention_mask.squeeze()
        }

In [15]:
train_ds = torch.load(Config.train_ds_path)
valid_ds = torch.load(Config.valid_ds_path)
test_ds = torch.load(Config.test_ds_path)

Config.train_len = len(train_ds)
Config.valid_len = len(valid_ds)
Config.test_len = len(test_ds)

In [16]:
model_config  = AutoConfig.from_pretrained(Config.model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(Config.model_checkpoint, config=model_config)

Downloading:   0%|          | 0.00/2.75G [00:00<?, ?B/s]

In [17]:
#with deepspeed.zero.Init():
model_config  = AutoConfig.from_pretrained(Config.model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(Config.model_checkpoint, config=model_config)
    
model.train()

if is_torch_tpu_available():
    WRAPPED_MODEL = xmp.MpModelWrapper(model)
elif Config.deepspeed_config:
    WRAPPED_MODEL = model
else:
    WRAPPED_MODEL = model.to("cuda")

data_collator = T2TDataCollator(tokenizer=tokenizer)

In [18]:
Config.epochs=4

if not Config.total_train_steps:
    Config.total_train_steps = (
                (Config.train_len // (Config.train_batch_size * max(1, Config.tpu_cores if Config.tpu_cores else 0)))
                // Config.gradient_accumulation_steps
                * float(Config.epochs)
            )

print("TOTAL TRAIN STEPS:", Config.total_train_steps)

Config.eval_steps = int(Config.total_train_steps * Config.eval_ratio)
print(f"Eval Ratio: {Config.eval_ratio}, Eval Steps: {Config.eval_steps}")

Config.logging_steps = int(Config.total_train_steps * Config.logging_ratio)
print(f"Logging Ratio: {Config.logging_ratio}, Logging Steps: {Config.logging_steps}")

Config.save_steps = int(Config.total_train_steps * Config.saving_ratio)
print(f"Save Ratio: {Config.saving_ratio}, Save Steps: {Config.save_steps}")

Config.warmup_steps = int(Config.total_train_steps * Config.warmup_ratio)
print(f"Warmup Ratio: {Config.warmup_ratio}, Warmup Steps: {Config.warmup_steps}")

Config.optimizer_warmup_init=False

TOTAL TRAIN STEPS: 100
Eval Ratio: 0.1, Eval Steps: 10
Logging Ratio: 0.01, Logging Steps: 1
Save Ratio: 0.2, Save Steps: 20
Warmup Ratio: 0.1, Warmup Steps: 10


In [19]:
proxy_valid_ds = valid_ds
proxy_test_ds = test_ds

valid_subset_len = Config.valid_batch_size * 3
test_subset_len = Config.test_batch_size * 2

#if is_torch_tpu_available():
proxy_valid_ds = torch.utils.data.Subset(valid_ds, range(valid_subset_len))
proxy_test_ds = torch.utils.data.Subset(test_ds, range(test_subset_len))


valid_data_loader = torch.utils.data.DataLoader(
    proxy_valid_ds,
    batch_size=Config.valid_batch_size,
    drop_last=Config.drop_last_valid,
    num_workers=Config.num_workers_valid,
)

test_data_loader = torch.utils.data.DataLoader(
    proxy_test_ds,
    batch_size=Config.test_batch_size,
    drop_last=Config.drop_last_test,
    num_workers=Config.num_workers_test,
)

In [20]:
print(len(valid_data_loader))
print(len(test_data_loader))

3
2


    from prettytable import PrettyTable

    def count_parameters(model):
        table = PrettyTable(["Modules", "Parameters"])
        total_params = 0
        for name, parameter in model.named_parameters():
            if not parameter.requires_grad: continue
            param = parameter.numel()
            table.add_row([name, param])
            total_params+=param
        print(table)
        print(f"Total Trainable Params: {total_params}")
        return total_params

    count_parameters(model)

### Rouge Before Training:

In [21]:
if Config.rouge_before_training:
    gen_kwargs = {
        "num_beams": Config.num_beams,
        "max_length":Config.max_output_len,
    }

    valid_results = compute_rouge(model, tokenizer, valid_data_loader, gen_kwargs=gen_kwargs, Config=Config)
    test_results = compute_rouge(model, tokenizer, test_data_loader, gen_kwargs=gen_kwargs, Config=Config)

    print("VALID RESULTS: ", valid_results)
    print("TEST RESULTS: ", test_results )

    for idx, batch in enumerate(test_ds):

        if is_torch_tpu_available():
            temp_batch={
                "input_ids": batch["input_ids"].unsqueeze(0),
                "attention_mask": batch["attention_mask"].unsqueeze(0),
            }
        else:
            temp_batch={
                "input_ids": batch["input_ids"].unsqueeze(0).to("cuda"),
                "attention_mask": batch["attention_mask"].unsqueeze(0).to("cuda"),
            }

        labels = batch["labels"]

        temp_batch.update(gen_kwargs)

        output_ids = model.generate(**temp_batch)
        decoded_input=tokenizer.decode(temp_batch["input_ids"].squeeze(), skip_special_tokens=False)
        decoded_output = tokenizer.decode(output_ids.squeeze(), skip_special_tokens=False)
        decoded_labels = tokenizer.decode(labels.squeeze(), skip_special_tokens=False)

        print(f"{'-'*60}Example-{idx}{'-'*60}")
        print("INPUT: ", decoded_input[:400], "."*10, decoded_input[-100:], "\n")
        print("LABEL: ", decoded_labels, "\n")
        print("PREDICTION: ", decoded_output, "\n")
        print("\n")

        if idx == 3:
            break

### My Custom TPU Trainer which implements serialized saving when limited host memory available.

In [22]:
from transformers.file_utils import (
    is_sagemaker_mp_enabled,
)
from transformers.trainer_utils import (
    ShardedDDPOption,
)
from transformers.deepspeed import (
    is_deepspeed_zero3_enabled
)
from transformers.modeling_utils import (
    PreTrainedModel,
    unwrap_model
)

class TPUTrainer(Trainer):
    
    def __init__(self, limited_host_mem=None, **kwargs):
        super().__init__(**kwargs)
        self.limited_host_mem=limited_host_mem
        
    def _tune_save_checkpoint(self):
        from ray import tune

        if not self.use_tune_checkpoints:
            return
        with tune.checkpoint_dir(step=self.state.global_step) as checkpoint_dir:
            output_dir = os.path.join(checkpoint_dir, f"{PREFIX_CHECKPOINT_DIR}-{self.state.global_step}")
            self.save_model(output_dir)
            if self.args.should_save:
                self.state.save_to_json(os.path.join(output_dir, "trainer_state.json"))
                torch.save(self.optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                torch.save(self.lr_scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
    
    def save_model(self, output_dir: Optional[str] = None, limited_host_mem=False):
        """
        Will save the model, so you can reload it using :obj:`from_pretrained()`.

        Will only save from the main process.
        """
        if self.limited_host_mem:
            limited_host_mem=self.limited_host_mem

        if output_dir is None:
            output_dir = self.args.output_dir

        if is_torch_tpu_available():
            self._save_tpu(output_dir, limited_host_mem)
        elif is_sagemaker_mp_enabled():
            # Calling the state_dict needs to be done on the wrapped model and on all processes.
            state_dict = self.model_wrapped.state_dict()
            if self.args.should_save:
                self._save(output_dir, state_dict=state_dict)
        elif (
            ShardedDDPOption.ZERO_DP_2 in self.args.sharded_ddp or ShardedDDPOption.ZERO_DP_3 in self.args.sharded_ddp
        ):
            state_dict = self.model.state_dict()

            if self.args.should_save:
                self._save(output_dir, state_dict=state_dict)
        elif self.deepspeed:

            # this takes care of everything as long as we aren't under zero3
            if self.args.should_save:
                self._save(output_dir)

            if is_deepspeed_zero3_enabled():
                # It's too complicated to try to override different places where the weights dump gets
                # saved, so since under zero3 the file is bogus, simply delete it. The user should
                # either user deepspeed checkpoint to resume or to recover full weights use
                # zero_to_fp32.py stored in the checkpoint.
                if self.args.should_save:
                    file = os.path.join(output_dir, WEIGHTS_NAME)
                    if os.path.isfile(file):
                        # logger.info(f"deepspeed zero3: removing {file}, see zero_to_fp32.py to recover weights")
                        os.remove(file)

                # now save the real model if stage3_gather_fp16_weights_on_model_save=True
                # if false it will not be saved.
                # This must be called on all ranks
                self.deepspeed.save_fp16_model(output_dir, WEIGHTS_NAME)

        elif self.args.should_save:
            self._save(output_dir)
    
    def _save_tpu(self, output_dir: Optional[str] = None, limited_host_mem=False):
        output_dir = output_dir if output_dir is not None else self.args.output_dir
        logger.info(f"Saving model checkpoint to {output_dir}")

        if xm.is_master_ordinal():
            os.makedirs(output_dir, exist_ok=True)
            torch.save(self.args, os.path.join(output_dir, "training_args.bin"))

        # Save a trained model and configuration using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        xm.rendezvous("saving_checkpoint")
        if not isinstance(self.model, PreTrainedModel):
            if isinstance(unwrap_model(self.model), PreTrainedModel):
                unwrap_model(self.model).save_pretrained(
                    output_dir,
                    save_config=self.args.should_save,
                    state_dict=self.model.state_dict(),
                    save_function=xm.xser if limited_host_mem else xm.save,
                )
            else:
                logger.info("Trainer.model is not a `PreTrainedModel`, only saving its state dict.")
                state_dict = self.model.state_dict()
                if limited_host_mem:
                    xm.xser(state_dict, os.path.join(output_dir, WEIGHTS_NAME))
                else:
                    xm.save(state_dict, os.path.join(output_dir, WEIGHTS_NAME))
        else:
            self.model.save_pretrained(output_dir, save_config=self.args.should_save,
                                       save_function=xm.xser if limited_host_mem else xm.save)
        if self.tokenizer is not None and self.args.should_save:
            self.tokenizer.save_pretrained(output_dir)

    def _save(self, output_dir: Optional[str] = None, state_dict=None):
        # If we are executing this function, we are the process zero, so we don't check for that.
        output_dir = output_dir if output_dir is not None else self.args.output_dir
        os.makedirs(output_dir, exist_ok=True)
        logger.info(f"Saving model checkpoint to {output_dir}")
        # Save a trained model and configuration using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        if not isinstance(self.model, PreTrainedModel):
            if isinstance(unwrap_model(self.model), PreTrainedModel):
                if state_dict is None:
                    state_dict = self.model.state_dict()
                unwrap_model(self.model).save_pretrained(output_dir, state_dict=state_dict)
            else:
                logger.info("Trainer.model is not a `PreTrainedModel`, only saving its state dict.")
                if state_dict is None:
                    state_dict = self.model.state_dict()
                torch.save(state_dict, os.path.join(output_dir, WEIGHTS_NAME))
        else:
            self.model.save_pretrained(output_dir, state_dict=state_dict)
        if self.tokenizer is not None:
            self.tokenizer.save_pretrained(output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(self.args, os.path.join(output_dir, "training_args.bin"))

In [23]:
def compute_metrics(eval_pred):
    rouge_metric = load_metric("rouge")
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    del predictions
    del decoded_preds
    del labels
    del decoded_labels
    del eval_pred
    del prediction_lens
    del rouge_metric
    gc.collect()
    
    return {k: round(v, 4) for k, v in result.items()}

In [24]:
def get_trainer(Config=Config, optimizer=None, lr_scheduler=None):
    training_args = TrainingArguments(
        max_steps=Config.total_train_steps,
        output_dir=Config.output_dir,
        num_train_epochs=Config.epochs,
        save_strategy=Config.save_strategy,
        evaluation_strategy=Config.evaluation_strategy,
        logging_dir=Config.logging_dir,
        eval_steps=Config.eval_steps,
        save_steps=Config.save_steps,
        logging_steps=Config.logging_steps,
        per_device_train_batch_size=Config.train_batch_size,
        per_device_eval_batch_size=Config.valid_batch_size,
        fp16=Config.fp16,
        deepspeed=Config.deepspeed_config,
        adafactor=Config.use_ada_factor
    )
    
    if is_torch_tpu_available():
        trainer = TPUTrainer(
            limited_host_mem=Config.limited_host_mem,
            model=model,
            args=training_args,
            compute_metrics=compute_metrics if Config.compute_metrics else None,
            data_collator=data_collator,
            train_dataset=train_ds,
            eval_dataset=valid_ds,
            optimizers=(optimizer, lr_scheduler),    
        )
    else:
        trainer = Trainer(
            model=model,
            args=training_args,
            compute_metrics=compute_metrics if Config.compute_metrics else None,
            data_collator=data_collator,
            train_dataset=train_ds,
            eval_dataset=valid_ds,
            optimizers=(optimizer, lr_scheduler),
        )
    return trainer


def get_optimizer_n_scheduler(model, Config=Config):
    
    no_decay = ["bias", "LayerNorm.weight"]
    
    grouped_params = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": Config.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        }
    ]

    optimizer = Adafactor(grouped_params, scale_parameter=Config.optimizer_scale_parameter, 
                          relative_step=Config.optimizer_relative_step, warmup_init=Config.optimizer_warmup_init,
                          lr=Config.lr)
    lr_scheduler = get_linear_schedule_with_warmup(optimizer,
                                                   num_training_steps=Config.total_train_steps,
                                                   num_warmup_steps =Config.warmup_steps)

    return optimizer, lr_scheduler

In [25]:
def _mp_fn_boiler_plate(index, Config=Config):
    
    device = xm.xla_device()
    model = WRAPPED_MODEL.to(device)

    optimizer, lr_scheduler = get_optimizer_n_scheduler(model, Config)
    
    print("Loading datasets... ", end="")

    trainer = get_trainer(Config, optimizer, lr_scheduler)
    trainer.place_model_on_device = False
    trainer.train()

    xser.save(trainer.model.state_dict(), Config.xser_save_path)
    trainer.model.config.to_json_file(Config.config_json_path)
    tokenizer.save_pretrained(Config.tokenizer_save_path)   

In [26]:
run(["rm", "-rf", Config.xser_save_path])

CompletedProcess(args=['rm', '-rf', 't5_large_xsum'], returncode=0)

In [27]:
Config.epochs=4

if not Config.total_train_steps:
    Config.total_train_steps = (
                (Config.train_len // (Config.train_batch_size * max(1, Config.tpu_cores if Config.tpu_cores else 0)))
                // Config.gradient_accumulation_steps
                * float(Config.epochs)
            )

print("TOTAL TRAIN STEPS:", Config.total_train_steps)

Config.eval_steps = int(Config.total_train_steps * Config.eval_ratio)
print(f"Eval Ratio: {Config.eval_ratio}, Eval Steps: {Config.eval_steps}")

Config.logging_steps = int(Config.total_train_steps * Config.logging_ratio)
print(f"Logging Ratio: {Config.logging_ratio}, Logging Steps: {Config.logging_steps}")

Config.save_steps = int(Config.total_train_steps * Config.saving_ratio)
print(f"Save Ratio: {Config.saving_ratio}, Save Steps: {Config.save_steps}")

Config.warmup_steps = int(Config.total_train_steps * Config.warmup_ratio)
print(f"Warmup Ratio: {Config.warmup_ratio}, Warmup Steps: {Config.warmup_steps}")

Config.optimizer_warmup_init=False

TOTAL TRAIN STEPS: 100
Eval Ratio: 0.1, Eval Steps: 10
Logging Ratio: 0.01, Logging Steps: 1
Save Ratio: 0.2, Save Steps: 20
Warmup Ratio: 0.1, Warmup Steps: 10


In [28]:
Config.compute_metrics = False

if is_torch_tpu_available():
    _mp_fn = partial(_mp_fn_boiler_plate, Config=Config)
    xmp.spawn(_mp_fn, start_method="fork")

    """
    ckpts_path = os.path.join(os.curdir, "results")
    model_ckpts = os.listdir(ckpts_path)
    ckpt_nums = [int(ckpt[11:]) for ckpt in model_ckpts]
    best_ckpt_num = max(ckpt_nums)
    best_ckpt_path = os.path.join(ckpts_path, f"checkpoint-{best_ckpt_num}")
    tokenizer.save_pretrained(best_ckpt_path)
    """

else:
    if Config.deepspeed_config and Config.use_ds_optimizer_scheduler:
        optimizer = None
        lr_scheduler = None
    else:
        optimizer, lr_scheduler = get_optimizer_n_scheduler(WRAPPED_MODEL, Config)
        #if Config.deepspeed_config
        
    #print(optimizer)
    trainer = get_trainer(Config, optimizer, lr_scheduler)
    #print(trainer.optimizer)
    trainer.train()
    
    #ckpts_path = os.path.join(os.curdir, "results")
    #model_ckpts = os.listdir(ckpts_path)
    #ckpt_nums = [int(ckpt[11:]) for ckpt in model_ckpts]
    #best_ckpt_num = max(ckpt_nums)
    #best_ckpt_path = os.path.join(ckpts_path, f"checkpoint-{best_ckpt_num}")
    #tokenizer.save_pretrained(best_ckpt_path)

[2021-11-03 07:30:38,152] [INFO] [distributed.py:47:init_distributed] Initializing torch distributed with backend: nccl


max_steps is given, it will override any value given in num_train_epochs
Using amp fp16 backend


[2021-11-03 07:30:43,797] [INFO] [logging.py:68:log_dist] [Rank 0] DeepSpeed info: version=0.5.6+426dd2b, git-hash=426dd2b, git-branch=master
[2021-11-03 07:30:45,218] [INFO] [logging.py:68:log_dist] [Rank 0] initializing deepspeed groups
[2021-11-03 07:30:45,219] [INFO] [logging.py:68:log_dist] [Rank 0] initializing deepspeed model parallel group with size 1
[2021-11-03 07:30:45,221] [INFO] [logging.py:68:log_dist] [Rank 0] initializing deepspeed expert parallel group with size 1
[2021-11-03 07:30:45,222] [INFO] [logging.py:68:log_dist] [Rank 0] creating expert data parallel process group with ranks: [0]
[2021-11-03 07:30:45,223] [INFO] [logging.py:68:log_dist] [Rank 0] creating expert parallel process group with ranks: [0]
[2021-11-03 07:30:45,295] [INFO] [engine.py:208:__init__] DeepSpeed Flops Profiler Enabled: False
[2021-11-03 07:30:45,296] [INFO] [engine.py:871:_configure_optimizer] Removing param_group that has no 'params' in the client Optimizer
[2021-11-03 07:30:45,297] [INFO

***** Running training *****
  Num examples = 204017
  Num Epochs = 1
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 1
  Total optimization steps = 100
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Loading extension module utils...
Time to load utils op: 14.274823904037476 seconds


wandb: Currently logged in as: prikmm (use `wandb login --relogin` to force relogin)


[2021-11-03 07:31:13,617] [INFO] [unfused_optimizer.py:275:_update_scale] Grad overflow on iteration: 0
[2021-11-03 07:31:13,618] [INFO] [unfused_optimizer.py:277:_update_scale] Reducing dynamic loss scale from 65536 to 32768.0
[2021-11-03 07:31:13,620] [INFO] [unfused_optimizer.py:202:step] [deepspeed] fp16 dynamic loss scale overflow! Skipping step. Attempted loss scale: 65536, reducing to 32768.0


Step,Training Loss,Validation Loss
10,8.828100,4.085938
20,1.210000,1.090820
30,1.264600,1.089844
40,1.118200,1.064453
50,1.153300,1.055664
60,1.091800,1.032227
70,0.971700,1.007812
80,1.129900,0.998535
90,1.004900,0.991211
100,0.857900,0.982422


[2021-11-03 07:31:15,035] [INFO] [unfused_optimizer.py:275:_update_scale] Grad overflow on iteration: 1
[2021-11-03 07:31:15,039] [INFO] [unfused_optimizer.py:277:_update_scale] Reducing dynamic loss scale from 32768.0 to 16384.0
[2021-11-03 07:31:15,040] [INFO] [unfused_optimizer.py:202:step] [deepspeed] fp16 dynamic loss scale overflow! Skipping step. Attempted loss scale: 32768.0, reducing to 16384.0
[2021-11-03 07:31:16,429] [INFO] [unfused_optimizer.py:275:_update_scale] Grad overflow on iteration: 2
[2021-11-03 07:31:16,430] [INFO] [unfused_optimizer.py:277:_update_scale] Reducing dynamic loss scale from 16384.0 to 8192.0
[2021-11-03 07:31:16,432] [INFO] [unfused_optimizer.py:202:step] [deepspeed] fp16 dynamic loss scale overflow! Skipping step. Attempted loss scale: 16384.0, reducing to 8192.0
[2021-11-03 07:31:17,822] [INFO] [unfused_optimizer.py:275:_update_scale] Grad overflow on iteration: 3
[2021-11-03 07:31:17,823] [INFO] [unfused_optimizer.py:277:_update_scale] Reducing d

***** Running Evaluation *****
  Num examples = 200
  Batch size = 10
***** Running Evaluation *****
  Num examples = 200
  Batch size = 10
***** Running Evaluation *****
  Num examples = 200
  Batch size = 10
***** Running Evaluation *****
  Num examples = 200
  Batch size = 10
***** Running Evaluation *****
  Num examples = 200
  Batch size = 10
***** Running Evaluation *****
  Num examples = 200
  Batch size = 10
***** Running Evaluation *****
  Num examples = 200
  Batch size = 10
***** Running Evaluation *****
  Num examples = 200
  Batch size = 10
***** Running Evaluation *****
  Num examples = 200
  Batch size = 10
***** Running Evaluation *****
  Num examples = 200
  Batch size = 10


Training completed. Do not forget to share your model on huggingface.co/models =)




In [ ]:
if is_torch_tpu_available():
    torch.cuda.empty_cache()

In [ ]:
if is_torch_tpu_available():
    tokenizer = AutoTokenizer.from_pretrained(Config.tokenizer_save_path,
                                              local_files_only=True,
                                              use_fast=Config.use_fast_tokenizer)
    
    model = AutoModelForSeq2SeqLM.from_pretrained(Config.model_checkpoint)
    state_dict = xser.load(Config.xser_save_path)
    model.load_state_dict(state_dict)
else:
    ckpts_path = os.path.join(os.curdir, "results")
    model_ckpts = os.listdir(ckpts_path)
    ckpt_nums = [int(ckpt[11:]) for ckpt in model_ckpts]
    best_ckpt_num = max(ckpt_nums)
    best_ckpt_path = os.path.join(ckpts_path, f"checkpoint-{best_ckpt_num}")
    tokenizer.save_pretrained(best_ckpt_path)
    model = AutoModelForSeq2SeqLM.from_pretrained(best_ckpt_path, local_files_only=True)
    model = model.to("cuda")

### Rouge After Training:

In [ ]:
gen_kwargs = {
    "num_beams": Config.num_beams,
    "max_length":Config.max_output_len,
}

valid_results = compute_rouge(model, tokenizer, valid_data_loader, gen_kwargs=gen_kwargs, Config=Config)
test_results = compute_rouge(model, tokenizer, test_data_loader, gen_kwargs=gen_kwargs, Config=Config)

print("VALID RESULTS: ", valid_results)
print("TEST RESULTS: ", test_results)

for idx, batch in enumerate(test_ds):
    
    if is_torch_tpu_available():
        temp_batch={
            "input_ids": batch["input_ids"].unsqueeze(0),
            "attention_mask": batch["attention_mask"].unsqueeze(0),
        }
    else:
        temp_batch={
            "input_ids": batch["input_ids"].unsqueeze(0).to("cuda"),
            "attention_mask": batch["attention_mask"].unsqueeze(0).to("cuda"),
        }
        
    labels = batch["labels"]
    
    temp_batch.update(gen_kwargs)

    output_ids = model.generate(**temp_batch)
    decoded_input=tokenizer.decode(temp_batch["input_ids"].squeeze(), skip_special_tokens=False)
    decoded_output = tokenizer.decode(output_ids.squeeze(), skip_special_tokens=False)
    decoded_labels = tokenizer.decode(labels.squeeze(), skip_special_tokens=False)

    print(f"{'-'*60}Example-{idx}{'-'*60}")
    print("INPUT: ", decoded_input[:400], "."*10, decoded_input[-100:], "\n")
    print("LABEL: ", decoded_labels, "\n")
    print("PREDICTION: ", decoded_output, "\n")
    print("\n")

    if idx == 3:
        break